In [176]:
import pandas as pd
import numpy as np
# 읽어올 엑셀 파일 지정
import difflib 
import re
filename_amf = './print/AMF_print.csv'
filename_smf = './print/SMF_print.csv'
filename_upf = './print/UPF_print.csv'



In [177]:
def split_values(df, column):
    df[column] = df[column].str.replace('DEDICATED', 'DEDICATE')
    df[column] = df[column].str.split(', ')
    df = df.explode(column).reset_index(drop=True)
    return df.fillna(method='ffill')

# df=split_values(df,'(통계) Columns')
# # df.loc[df['(통계) Columns']=='DEDICATED_ATTEMPT','(통계) Columns'] = 'DEDICATED_ATTEMPT'
# df

def find_txt(txt_array,a):
    sell_index = np.where(txt_array == a+" ")[0]
    fruite_index = np.where(txt_array == '명령어 포맷 ')[0]
    
    # "sell"과 "fruite"이 연속으로 나오는 인덱스 찾기
    sell_fruite_index = int(np.intersect1d(sell_index + 1, fruite_index))
    
    # # "sell", "fruite", "apple" 사이의 값들 찾기
    fruite_index = np.where(txt_array[sell_fruite_index:] == '입력 파라미터 설명 ')[0][0] 
    
    result = txt_array[sell_fruite_index+3:sell_fruite_index+fruite_index] ## +3 -> 명령어 설명 부분만 추출하기
    return result

def find_txt_v2(txt_array,a):
    sell_index = np.where(txt_array == a+" ")[0]
    fruite_index = np.where(txt_array == '명령어 포맷 ')[0]
    
    # "sell"과 "fruite"이 연속으로 나오는 인덱스 찾기
    sell_fruite_index = int(np.intersect1d(sell_index + 1, fruite_index))
    
    # # "sell", "fruite", "apple" 사이의 값들 찾기
    fruite_index = np.where(txt_array[sell_fruite_index:] == '명령어 포맷 ')[0][1] 
    
    result = txt_array[sell_fruite_index+3:sell_fruite_index+fruite_index] ## +3 -> 명령어 설명 부분만 추출하기
    return result



Data Load from ./print/~~~~
 - 문서 관련 폴더 : ./print 

In [178]:
df_upf = pd.read_csv(filename_upf,header=1).fillna(method='ffill')
df_smf = pd.read_csv(filename_smf,header=0).fillna(method='ffill')
df_amf  =pd.read_csv(filename_amf,header=1).fillna(method='ffill')

In [179]:
df_amf

,Family ID,Family Name,Family Display Name,Family Description,Index Name,Index Range,Index Description,Type Name,Data Type,Min/Max,Unit,Counter Type,Type Description,Granularity Formula,Caculation Type,"Hourly, Daily, Monthly Formula",Retrieve Command,Measure Command
0,451.0,MS_MS,MS,CURRENT MEASUREMENT OF MOBILE SUBSCRIBER,NODE,ALL/DBC/DBC0~DBC24,이 파라미터는 NODE 이름이다.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RTRV-MEAS-MS,MEAS-MS
1,451.0,MS_MS,MS,CURRENT MEASUREMENT OF MOBILE SUBSCRIBER,NODE,ALL/DBC/DBC0~DBC24,이 파라미터는 NODE 이름이다.,CUR_REG,long long,"0~2,147,483,774",count,CC,이 파라미터는 가입자의 현재 등록 상태가 Registered인 가입자 수이다.,None,SUM,sum,RTRV-MEAS-MS,MEAS-MS
2,451.0,MS_MS,MS,CURRENT MEASUREMENT OF MOBILE SUBSCRIBER,NODE,ALL/DBC/DBC0~DBC24,이 파라미터는 NODE 이름이다.,CUR_DEREG,long long,"0~2,147,483,774",count,CC,이 파라미터는 가입자의 현재 등록 상태가 Deregistered인 가입자 수이다.,None,SUM,sum,RTRV-MEAS-MS,MEAS-MS
3,451.0,MS_MS,MS,CURRENT MEASUREMENT OF MOBILE SUBSCRIBER,NODE,ALL/DBC/DBC0~DBC24,이 파라미터는 NODE 이름이다.,CUR_TOTAL,long long,"0~2,147,483,774",count,CC,이 파라미터는 등록된 총가입자 수이다.,None,SUM,sum,RTRV-MEAS-MS,MEAS-MS
4,451.0,MS_MS,MS,CURRENT MEASUREMENT OF MOBILE SUBSCRIBER,NODE,ALL/DBC/DBC0~DBC24,이 파라미터는 NODE 이름이다.,CUR_CM_IDLE,long long,"0~2,147,483,774",count,CC,이 파라미터는 CM_STATE가 CM_IDLE 상태인 가입자의 수이다.,None,SUM,sum,RTRV-MEAS-MS,MEAS-MS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11502,734.0,SMSNAS_DL,SMSNASDL,MEASUREMENT OF SMS OVER NAS PER DOWNLINK,AMCS,ALL/0~15,이 파라미터는 AMCS ID이다.,UE_IDLE,long long,"0~2,147,483,774",count,CC,이 파라미터는 UE가 idle 상태이어서 실패한 횟수이다.,None,SUM,sum,RTRV-MEAS-SMSNAS,MEAS-SMSNAS
11503,734.0,SMSNAS_DL,SMSNASDL,MEASUREMENT OF SMS OVER NAS PER DOWNLINK,AMCS,ALL/0~15,이 파라미터는 AMCS ID이다.,LAST_IND,long long,"0~2,147,483,774",count,CC,이 파라미터는 lastMsgIndication이 true인 Namf_Communic...,None,SUM,sum,RTRV-MEAS-SMSNAS,MEAS-SMSNAS
11504,734.0,SMSNAS_DL,SMSNASDL,MEASUREMENT OF SMS OVER NAS PER DOWNLINK,AMCS,ALL/0~15,이 파라미터는 AMCS ID이다.,SYS_FLT,long long,"0~2,147,483,774",count,CC,이 파라미터는 시스템 실패로 인해 실패한 횟수이다.,None,SUM,sum,RTRV-MEAS-SMSNAS,MEAS-SMSNAS
11505,734.0,SMSNAS_DL,SMSNASDL,MEASUREMENT OF SMS OVER NAS PER DOWNLINK,AMCS,ALL/0~15,이 파라미터는 AMCS ID이다.,ETC_ERR,long long,"0~2,147,483,774",count,CC,이 파라미터는 기타 사유로 인해 실패한 횟수이다.,None,SUM,sum,RTRV-MEAS-SMSNAS,MEAS-SMSNAS


In [180]:

### PARSER
file = open("./print/vSMF 입력문 설명서.txt", "r",encoding='UTF-8')
# 파일 내용 읽기
content = file.read()
# 파일 닫기
file.close()
content=content.splitlines()

#입력문 -> Measure command -> ['MEAS-LOAD','명령어 포멧'] ~ ['입력 파라미터 설명']
#입력문 -> Retrieve Command -> ['RTRV-MEAS-LOAD', '명령어 포멧'] ~ ['입력 파리미터 설명']



In [181]:
##### DB name , statistic / 
#### True 읽고 답하기
ad_result = pd.read_csv("./test_ai_results_202405241637.csv").iloc[:,1:]
ad_result = ad_result.loc[ad_result['detection']==1,:]
ad_result


,original_datetime,db_name,nf_name,table_name,port,statistic,statistic_value,score,abnormality,detection,created_at,ratio,threshold
19,2024-05-24 16:10:00,GR_AMF21_5M,GURO_AMF21_SS,SR|IPLMN,_IPLMN45008,UE_INIT_ATTEMPT(count),170522.0,3.074552e-03,1064.558844,1,2024-05-24 16:15:17,NaN,2.888100e-04
33,2024-05-24 16:10:00,GR_SMF01_5M,GURO_SMF01_SS,PFCP|PFMC,_PEER39.19.46.1 (GURO_UPF06_SS),AS_S_REQ_RX_C_RATIO(%),0.0,1.163727e-10,1000.000000,1,2024-05-24 16:15:51,NaN,0.000000e+00
40,2024-05-24 16:10:00,GR_SMF01_5M,GURO_SMF01_SS,PFCP|PFMC,_PEER39.18.145.1 (BUSN_UPF04_SS),AS_S_REQ_RX_ATTEMPT(count),0.0,3.353850e-11,1000.000000,1,2024-05-24 16:16:06,NaN,0.000000e+00
46,2024-05-24 16:10:00,GR_SMF01_5M,GURO_SMF01_SS,PFCPN|PFMC,_PIC0,AS_S_REQ_RX_C_RATIO(%),0.0,4.683480e-11,1000.000000,1,2024-05-24 16:16:16,NaN,0.000000e+00
49,2024-05-24 16:10:00,GR_SMF01_5M,GURO_SMF01_SS,PFCP|PFMC,_PEER39.18.27.1 (HEHA_UPF05_SS),AS_S_REQ_RX_C_RATIO(%),0.0,3.353850e-11,1000.000000,1,2024-05-24 16:16:19,NaN,0.000000e+00
62,2024-05-24 16:10:00,GR_SMF01_5M,GURO_SMF01_SS,PFCPN|PFMC,_PIC2,AS_S_REQ_RX_C_RATIO(%),0.0,4.683480e-11,1000.000000,1,2024-05-24 16:16:34,NaN,0.000000e+00
76,2024-05-24 16:10:00,GR_SMF01_5M,GURO_SMF01_SS,PFCP|PFMC,_PEER39.18.25.1 (HEHA_UPF03_SS),AS_S_REQ_RX_C_RATIO(%),0.0,3.353850e-11,1000.000000,1,2024-05-24 16:16:56,NaN,0.000000e+00
93,2024-05-24 16:10:00,GR_SMF01_5M,GURO_SMF01_SS,PFCP|PFMC,_PEER39.18.152.1 (SEBS_UPF01_SS),AS_S_REQ_RX_C_RATIO(%),0.0,3.128800e-12,1000.000000,1,2024-05-24 16:17:21,NaN,0.000000e+00
102,2024-05-24 16:10:00,GR_SMF01_5M,GURO_SMF01_SS,PFCP|PFMC,_PEER39.18.28.1 (HEHA_UPF06_SS),AS_S_REQ_RX_C_RATIO(%),0.0,1.289574e-10,1000.000000,1,2024-05-24 16:17:28,NaN,0.000000e+00
113,2024-05-24 16:10:00,GR_SMF01_5M,GURO_SMF01_SS,GTPC|TMMC,_PEER39.18.137.33,CRT_BEAR_ATTEMPT(count),398.0,1.092913e-03,125.423194,1,2024-05-24 16:17:45,NaN,8.713800e-04



txt_array = np.array(content)
# print(txt_array[10090:100100])
# "sell"과 "fruite"의 인덱스 찾기

sell_index = np.where(txt_array == 'RTRV-MEAS-LOAD ')[0]
fruite_index = np.where(txt_array == '명령어 포맷 ')[0]
print(sell_index,fruite_index)
# "sell"과 "fruite"이 연속으로 나오는 인덱스 찾기
sell_fruite_index = int(np.intersect1d(sell_index + 1, fruite_index))
# print(sell_fruite_index)
# print(txt_array[sell_fruite_index:][6:7])
# "sell", "fruite", "apple" 사이의 값들 찾기
fruite_index = np.where(txt_array[sell_fruite_index:] == '입력 파라미터 설명 ')[0][0]
print(fruite_index)
result = txt_array[sell_fruite_index-1:sell_fruite_index+fruite_index]


print("Values between 'sell', 'fruite', and 'apple':", result)

In [182]:
import re
import numpy as np
import difflib

file = open("./print/vUPF 출력문 설명서.txt", "r", encoding='UTF-8')
# 파일 내용 읽기
content2 = file.read()
# 파일 닫기
file.close()
content2 = content2.splitlines()
txt_array2 = np.array(content2)


a = "HOURLY REPORT - MEASUREMENT OF PACKET USER DATA"

def find_near_string(result_list, feature):
    score = 0
    best_index = 0
    # best_feature = ''
    for index,i in enumerate(result_list):
        similarity = difflib.SequenceMatcher(None, i, feature).ratio()
        # print(similarity)
        if similarity > score:
            score = similarity
            best_feature = i
            best_index = index
    print(score)
    
    return best_feature, best_index



In [217]:
##(txt_array2,descrip["Family Description"].values[0],descrip["Type Name"].values[0],content2)
## ㅇ원문 , descrip["Family Description"]  , 컬럼명, 
def find_feature_description_old(txt_array2, a, feature,content2):

    best_feature, sell_index=find_near_string(content2,a)
    txt_array2 = np.array(content2)
    # print("index상태확인---------------")
    # print(content2[sell_index])
    # print(txt_array2[sell_index])
    # print("----------------------")
    if sell_index > 0:
        
        fruite_index = np.where(txt_array2 == '   SYSNAME YYYY-MM-DD DAYOFWEEK HH:MM:SS ')[0]
        # "sell"과 "fruite"이 연속으로 나오는 인덱스 찾기
    
        sell_index=np.array([sell_index])
        # print(fruite_index,sell_index)
        print("array에서 FD찾기",txt_array2[sell_index[0]-4:sell_index[0]+2])
        try:
            sell_fruite_index = int(np.intersect1d(fruite_index,[sell_index-1] ))
            fruite_index = np.where(txt_array2[sell_fruite_index:] == '   SYSNAME YYYY-MM-DD DAYOFWEEK HH:MM:SS ')[0][1]
            result_list = content2[sell_fruite_index:sell_fruite_index+fruite_index]
        except Exception as e:
            print(e)
            sell_fruite_index = int(np.intersect1d(fruite_index,[sell_index+5] ))
            fruite_index = np.where(txt_array2[sell_fruite_index:] == '   SYSNAME YYYY-MM-DD DAYOFWEEK HH:MM:SS ')[0][1]
            result_list = content2[sell_fruite_index:sell_fruite_index+fruite_index]
        # # "sell", "fruite", "apple" 사이의 값들 찾기
        _ , result_index = find_near_string(result_list, feature)
        
        
        if len(result_list[result_index+3]) >= 14:
            # print((result_list[result_index:result_index+3]))
            return "".join(result_list[result_index:result_index+3])
        
        # print((result_list[result_index:result_index+2]))    
        return result_list[result_index:result_index+1]

##(txt_array2,descrip["Family Description"].values[0],descrip["Type Name"].values[0],content2)
## ㅇ원문 , descrip["Family Description"]  , 컬럼명, 
def find_feature_description(content, col, command):
    
    col = col[0].split("_")
    length_col = len(col)
    sell_index = []
    sell_index2= []
    
    txt_array = np.array(content)
    tmp_txt = find_txt_v2(txt_array,command)
    # print(tmp_txt)
    
    empty = False
    for i in range(length_col):
        
        first= "_".join(col[:len(col)-i-1])
        if len(first) == 0 : ## 첫번째 블럭에서는 겹치는 특징이 없을 수 있음
            empty = True
            break
        fruit_index = np.where(tmp_txt == first+" ")[0]
        print("fruit_index",first,fruit_index)
        if len(fruit_index) != 0:
            sell_index=np.array([fruit_index[0]+1])    
            sell_index=fruit_index[0]+1
            break
        
    for i in range(length_col):
        sec= "_".join(col[len(col)-i-1:])
        fruit_index2 = np.where(tmp_txt == sec+" ")[0]
        # "sell"과 "fruite"이 연속으로 나오는 인덱스 찾기
        print("fruit_index2",sec,fruit_index2)
        if len(fruit_index2) != 0:
            sell_index2=np.array([fruit_index2[0]+1])
            sell_index2=fruit_index2[0]+1
            break
    if sell_index2 == []:
        return tmp_txt[sell_index]
            


    if empty:
        print( str(first) + tmp_txt[sell_index2])
        return str(first) + tmp_txt[sell_index2]
    else:
        print( tmp_txt[sell_index] + tmp_txt[sell_index2])
        return tmp_txt[sell_index] + tmp_txt[sell_index2]

In [187]:
nf_lists = {'UPF': ['BS_UPF01_5M', 'BS_UPF03_5M', 'DG_UPF01_5M', 'DJ_UPF01_5M', 'DJ_UPF03_5M', 'GJ_UPF01_5M', 'GR_UPF01_5M', 'GR_UPF03_5M', 'GR_UPF05_5M', 'GR_UPF06_5M', 'HH_UPF01_5M', 'HH_UPF03_5M', 'HH_UPF04_5M', 'HH_UPF05_5M', 'HH_UPF06_5M'], 'SMF': ['BS_SMF02_5M', 'BS_SMF03_5M', 'DJ_SMF01_5M', 'DJ_SMF03_5M', 'GR_SMF01_5M', 'GR_SMF04_5M', 'HH_SMF01_5M', 'HH_SMF04_5M'], 'AMF': ['BS_AMF21_5M', 'BS_AMF22_5M', 'DJ_AMF21_5M', 'DJ_AMF22_5M', 'GR_AMF21_5M', 'GR_AMF22_5M', 'HH_AMF21_5M', 'HH_AMF22_5M']}

In [227]:
result = pd.DataFrame()
for nf, db_list in nf_lists.items():
    ad_result1 = ad_result.loc[:,['original_datetime','db_name','table_name',"statistic","port","statistic_value","abnormality"]] 
    for db in db_list:        
        tmp_df = ad_result1.loc[ad_result1["db_name"]==db,:]
        if tmp_df.empty:
            # print('DataFrame is empty!')
            continue
        print(tmp_df.head)
        table_list = ad_result.loc[ad_result['db_name']==db,"table_name"].unique()
        table_list_org = ad_result.loc[ad_result['db_name']==db,"table_name"].unique()
        print(table_list_org)
        print(table_list)
        # break
        if nf == "UPF":
            df_upf["NF"] = nf
            ex = df_upf
            file = open("./print/vUPF 입력문 설명서.txt", "r",encoding='UTF-8')
            # 파일 내용 읽기
            content = file.read()
            # 파일 닫기
            file.close()
            content=content.splitlines()

            file = open("./print/vUPF 출력문 설명서.txt", "r",encoding='UTF-8')
            # 파일 내용 읽기
            content2 = file.read()
            # 파일 닫기
            file.close()
            content2=content2.splitlines()

        elif nf == "AMF":
            df_amf["NF"] = nf
            ex = df_amf
            file = open("./print/vAMF 입력문 설명서.txt", "r",encoding='UTF-8')
            # 파일 내용 읽기
            content = file.read()
            # 파일 닫기
            file.close()

            content=content.splitlines()

            file = open("./print/vAMF 출력문 설명서.txt", "r",encoding='UTF-8')
            # 파일 내용 읽기
            content2 = file.read()
            # 파일 닫기
            file.close()
            content2=content2.splitlines()

        elif nf =="SMF":
            df_smf["NF"] = nf
            ex = df_smf
            file = open("./print/vSMF 입력문 설명서.txt", "r",encoding='UTF-8')
            # 파일 내용 읽기
            content = file.read()
            # 파일 닫기
            file.close()

            content=content.splitlines()    # print(i)

            file = open("./print/vSMF 출력문 설명서.txt", "r",encoding='UTF-8')
            # 파일 내용 읽기
            content2 = file.read()
            # 파일 닫기
            file.close()
            content2=content2.splitlines()


        txt_array = np.array(content)
        txt_array2 = np.array(content2)
        print(nf ,np.shape(txt_array))
        
        
        for j,q in zip(table_list,table_list_org):
            # print(j,q)
            tmp_table_df = tmp_df.loc[tmp_df['table_name']==q,['original_datetime','db_name','table_name',"statistic","port","statistic_value","abnormality"]].reset_index(drop=True)
            col_list = tmp_table_df.loc[tmp_table_df['table_name']==q,"statistic"].unique()

            
            for a,p in zip(tmp_table_df.values,col_list):
                # print(a,p)
                datetime,db,tab,col,port,val,abn = [element.split() if isinstance(element, str) else element for element in a]
                
                print (db,tab,col,port,val,abn)
                print(p)
                ##column 명 특수기호 포함 찾기 ## count,ratio,KB 등등 빼고 동일한거 찾기
                ex["Type Name"] = ex["Type Name"].str.replace(r'\(.*?\)', '')
                p = re.sub(r'\(.*?\)', '', p)
                j = j.replace('|', '_')
                # print(j,p)
                descrip = ex.loc[(ex["Family Name"]==j) & (ex["Type Name"]==p), ["NF","Family Name","Type Name","Family Description", "Type Description","Retrieve Command", "Measure Command"]].reset_index(drop=True)
                print(descrip["Family Name"].values)
                print(descrip["Retrieve Command"].values)
                #_마지막 것을 빼고 조합을 해서 입력문을 찾으면  SD _REQ 의미를 찾음 _> Column Description에 추가
                descrip["DATETIME"] = [datetime[0]+" "+datetime[1]]
                descrip["NF"] = db
                descrip["PORT"] = port if len(port) ==1 else [port[0] +" "+port[1]]
                descrip["VALUE"] = val
                descrip["ABNORMALITY"] = abn
                description_result = list([find_feature_description(content,descrip["Type Name"].values,descrip["Measure Command"].values[0])] )
                print("---------------------------------")
                print(description_result[0],len(description_result[0]))
                Retrieve_result = find_txt(txt_array,descrip["Retrieve Command"].values[0])
                Measure_result = find_txt(txt_array,descrip["Measure Command"].values[0])
                descrip["Type Description Result"]= description_result if len(description_result[0]) != 0 else descrip["Type Description"].values[0]

                descrip["Retrieve Command Result"]= "".join(list(Retrieve_result))
                descrip["Measure Command Result"]=  "".join(list(Measure_result))
                result = pd.concat([result,descrip],axis=0)




    # print(result)

<bound method NDFrame.head of        original_datetime      db_name     table_name  \
182  2024-05-24 16:10:00  GR_UPF01_5M      PPORT|PKT   
206  2024-05-24 16:10:00  GR_UPF01_5M     PPORT|DROP   
213  2024-05-24 16:10:00  GR_UPF01_5M     PPORT|DROP   
230  2024-05-24 16:10:00  GR_UPF01_5M     PPORT|DROP   
245  2024-05-24 16:10:00  GR_UPF01_5M     PPORT|DROP   
253  2024-05-24 16:10:00  GR_UPF01_5M     PPORT|DROP   
280  2024-05-24 16:10:00  GR_UPF01_5M     PPORT|DROP   
288  2024-05-24 16:10:00  GR_UPF01_5M      PPORT|PKT   
322  2024-05-24 16:10:00  GR_UPF01_5M     PPORT|DROP   
330  2024-05-24 16:10:00  GR_UPF01_5M     PPORT|DROP   
347  2024-05-24 16:10:00  GR_UPF01_5M     PPORT|DROP   
350  2024-05-24 16:10:00  GR_UPF01_5M     PPORT|DROP   
354  2024-05-24 16:10:00  GR_UPF01_5M     PFCPN|PFMC   
365  2024-05-24 16:10:00  GR_UPF01_5M     PPORT|DROP   
373  2024-05-24 16:10:00  GR_UPF01_5M     PPORT|DROP   
374  2024-05-24 16:10:00  GR_UPF01_5M  PPORT|DROPDTL   
386  2024-05-24 16

/tmp/ipykernel_318519/2053815547.py:87: FutureWarning: The default value of regex will change from True to False in a future version.
  ex["Type Name"] = ex["Type Name"].str.replace(r'\(.*?\)', '')
/tmp/ipykernel_318519/2053815547.py:87: FutureWarning: The default value of regex will change from True to False in a future version.
  ex["Type Name"] = ex["Type Name"].str.replace(r'\(.*?\)', '')
/tmp/ipykernel_318519/753653368.py:75: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if sell_index2 == []:
/tmp/ipykernel_318519/2053815547.py:87: FutureWarning: The default value of regex will change from True to False in a future version.
  ex["Type Name"] = ex["Type Name"].str.replace(r'\(.*?\)', '')


fruit_index RX_L2 []
fruit_index RX []
fruit_index2 DROP [730]
해당 포트에서 drop 된 총 패킷 수이다. 
---------------------------------
해당 포트에서 drop 된 총 패킷 수이다.  25
['GR_UPF01_5M'] ['PPORT|DROP'] ['RX_L2_DROP(count)'] ['_PPC9_0_V4'] 0.0 1000.0
RX_L3_DROP(count)
['PPORT_DROP']
['RTRV-MEAS-PPORT']
fruit_index RX_L3 []
fruit_index RX []
fruit_index2 DROP [730]
해당 포트에서 drop 된 총 패킷 수이다. 
---------------------------------
해당 포트에서 drop 된 총 패킷 수이다.  25
['GR_UPF01_5M'] ['PFCPN|PFMC'] ['AS_S_REQ_RX_ATTEMPT(count)'] ['_PIC0'] 0.0 103.5633534485214
AS_S_REQ_RX_ATTEMPT(count)
['PFCPN_PFMC']
['RTRV-MEAS-PFCPN']
fruit_index AS_S_REQ_RX []
fruit_index AS_S_REQ [173]
fruit_index2 ATTEMPT [347 465 572 682 780 878]
이 파라미터는 Sx Association Setup Request Message 의 송신 수신 개수이다. 이 파라미터는 PFCP 메시지별 수신 및 처리 시도 수이다. 
---------------------------------
이 파라미터는 Sx Association Setup Request Message 의 송신 수신 개수이다. 이 파라미터는 PFCP 메시지별 수신 및 처리 시도 수이다.  93
['GR_UPF01_5M'] ['PPORT|DROPDTL'] ['RX_L4BAD_HDR(count)'] ['_PPC11_0_V6'] 20.0 577

/tmp/ipykernel_318519/753653368.py:75: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if sell_index2 == []:
/tmp/ipykernel_318519/2053815547.py:87: FutureWarning: The default value of regex will change from True to False in a future version.
  ex["Type Name"] = ex["Type Name"].str.replace(r'\(.*?\)', '')
/tmp/ipykernel_318519/753653368.py:75: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if sell_index2 == []:
/tmp/ipykernel_318519/2053815547.py:87: FutureWarning: The default value of regex will change from True to False in a future version.
  ex["Type Name"] = ex["Type Name"].str.replace(r'\(.*?\)', '')
/tmp/ipykernel_318519/753653368.py:75: DeprecationWarning: The truth value of an empty array is ambiguou

fruit_index RX_L4BAD []
fruit_index RX []
fruit_index2 HDR []
fruit_index2 L4BAD_HDR [441]
Port 에서 수신된 Packet 중 L4 BAD Header 에 의해 drop 된 packet 수이다. 예를 
---------------------------------
Port 에서 수신된 Packet 중 L4 BAD Header 에 의해 drop 된 packet 수이다. 예를  62
<bound method NDFrame.head of        original_datetime      db_name    table_name  \
33   2024-05-24 16:10:00  GR_SMF01_5M     PFCP|PFMC   
40   2024-05-24 16:10:00  GR_SMF01_5M     PFCP|PFMC   
46   2024-05-24 16:10:00  GR_SMF01_5M    PFCPN|PFMC   
49   2024-05-24 16:10:00  GR_SMF01_5M     PFCP|PFMC   
62   2024-05-24 16:10:00  GR_SMF01_5M    PFCPN|PFMC   
76   2024-05-24 16:10:00  GR_SMF01_5M     PFCP|PFMC   
93   2024-05-24 16:10:00  GR_SMF01_5M     PFCP|PFMC   
102  2024-05-24 16:10:00  GR_SMF01_5M     PFCP|PFMC   
113  2024-05-24 16:10:00  GR_SMF01_5M     GTPC|TMMC   
136  2024-05-24 16:10:00  GR_SMF01_5M     GTPC|TMMC   
142  2024-05-24 16:10:00  GR_SMF01_5M  COSVC|AMFCOM   
188  2024-05-24 16:10:00  GR_SMF01_5M     GTPC|TMMC   
1

...

/tmp/ipykernel_318519/2053815547.py:87: FutureWarning: The default value of regex will change from True to False in a future version.
  ex["Type Name"] = ex["Type Name"].str.replace(r'\(.*?\)', '')
/tmp/ipykernel_318519/753653368.py:75: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if sell_index2 == []:
/tmp/ipykernel_318519/2053815547.py:87: FutureWarning: The default value of regex will change from True to False in a future version.
  ex["Type Name"] = ex["Type Name"].str.replace(r'\(.*?\)', '')


fruit_index UE_INIT []
fruit_index UE []
fruit_index2 ATTEMPT [240 355 458 575 678 781]
Service Request 시도 수이다. 
---------------------------------
Service Request 시도 수이다.  24
['GR_AMF21_5M'] ['PDUSMF|SE'] ['INIT_ATTEMPT(count)'] ['_IP39.18.11.3'] 278.99999999999994 419.4123067725517
INIT_ATTEMPT(count)
['PDUSMF_SE']
['RTRV-MEAS-PDUSMF']
fruit_index INIT []
fruit_index2 ATTEMPT [176 316 442 577 697 791 930]
이 파라미터는 PDUSession 시도 횟수이다. 
---------------------------------
이 파라미터는 PDUSession 시도 횟수이다.  28
['GR_AMF21_5M'] ['PDUSMF|SM'] ['UE_ATTEMPT(count)'] ['_IP39.18.11.2'] 270.0 239.7231256321534
UE_ATTEMPT(count)
['PDUSMF_SM']
['RTRV-MEAS-PDUSMF']


/tmp/ipykernel_318519/753653368.py:75: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if sell_index2 == []:
/tmp/ipykernel_318519/2053815547.py:87: FutureWarning: The default value of regex will change from True to False in a future version.
  ex["Type Name"] = ex["Type Name"].str.replace(r'\(.*?\)', '')
/tmp/ipykernel_318519/753653368.py:75: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if sell_index2 == []:
/tmp/ipykernel_318519/2053815547.py:87: FutureWarning: The default value of regex will change from True to False in a future version.
  ex["Type Name"] = ex["Type Name"].str.replace(r'\(.*?\)', '')


fruit_index UE []
fruit_index2 ATTEMPT [176 316 442 577 697 791 930]
이 파라미터는 PDUSession 시도 횟수이다. 
---------------------------------
이 파라미터는 PDUSession 시도 횟수이다.  28
['GR_AMF21_5M'] ['COSVC|UDMUECM'] ['DEREGNOTI_ATTEMPT(count)'] ['_PEER10.124.65.67'] 2214.0 100.8582593864858
DEREGNOTI_ATTEMPT(count)
['COSVC_UDMUECM']
['RTRV-MEAS-COSVC']
fruit_index DEREGNOTI []
fruit_index2 ATTEMPT [ 203  288  375  472  557  642  729  814  901  998 1083 1168 1255 1340
 1427 1524 1609]
이 파라미터는 호출된 Service Request 메시지와 Service Notification 메시지 
---------------------------------
이 파라미터는 호출된 Service Request 메시지와 Service Notification 메시지  58
['GR_AMF21_5M'] ['COSVC|UEAUTH'] ['AUTH_ATTEMPT(count)'] ['_PEER10.124.62.54'] 249.0 182.95412106917968
AUTH_ATTEMPT(count)
['COSVC_UEAUTH']
['RTRV-MEAS-COSVC']
fruit_index AUTH []
fruit_index2 ATTEMPT [ 203  288  375  472  557  642  729  814  901  998 1083 1168 1255 1340
 1427 1524 1609]
이 파라미터는 호출된 Service Request 메시지와 Service Notification 메시지 
-------------------------

/tmp/ipykernel_318519/753653368.py:75: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if sell_index2 == []:
/tmp/ipykernel_318519/2053815547.py:87: FutureWarning: The default value of regex will change from True to False in a future version.
  ex["Type Name"] = ex["Type Name"].str.replace(r'\(.*?\)', '')
/tmp/ipykernel_318519/753653368.py:75: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if sell_index2 == []:


In [228]:
result.columns
result = result[['DATETIME','NF', 'Family Name', 'Type Name','PORT',
       'VALUE', 'ABNORMALITY' , 'Family Description',
       'Type Description Result', 'Retrieve Command', 'Measure Command', 
       'Retrieve Command Result', 'Measure Command Result']]
result.to_csv("Test.csv",header = True, index = False, encoding='utf-8-sig')
result.to_csv("test.xlsx")


In [ ]:
port